### Libraries

In [ ]:
import re
from os import path
import pandas as pd
from tqdm import tqdm_notebook

### Datasets GunControl, GayMarriage and RestaurantReviews

In [ ]:
def reviews_options_datasets(arquivo, name_id):
    file = open("../Datasets/Originais/" + arquivo, "r")
    linhas = []
    
    for linha in file:
        if not re.search("MouseUp", linha) == None:
            inicio = min(re.search("MouseUp", linha).span()[0], re.search("KeyDown", linha).span()[0], re.search("KeyUp", linha).span()[0])
            ponto_virgula = re.search(";", linha).span()[0]
            while ponto_virgula < inicio:
                linha = re.sub(';', ' ', linha, 1)
                ponto_virgula = re.search(";", linha).span()[0]
        linhas.append(linha)
        
    
    for i in range(0,len(linhas)):
        linhas[i] = linhas[i].replace("\t",";")
        linhas[i] = linhas[i].replace("\n","")
        linhas[i] = linhas[i].split(";")
        
    colunas_max = 0
    for i in linhas:
        colunas_max = max(len(i), colunas_max)
    
    df = pd.DataFrame(linhas[1:], columns=pd.concat([pd.Series(linhas[0]),pd.Series(range(0,colunas_max-len(linhas[0])))]))
    df["User_Label"] = df["AccessKey"]
    df["Sample_Label"] = name_id + "|" + df["AccessKey"] + "|" + df["Task"]  
    df.to_csv("../Datasets/Tratados/" + arquivo, index=False)
    
    return df

df1 = reviews_options_datasets("GunControl_400.csv", "gun")
df2 = reviews_options_datasets("GayMarriage_400.csv", "gay")
df3 = reviews_options_datasets("ReviewAMT_500_t.csv", "rest")

In [ ]:
# GunControl Dataset
df1.shape
df1.head()

In [ ]:
# GayMarriage Dataset
df2.shape
df2.head()

In [ ]:
# RestaurantReview Dataset
df3.shape
df3.head()

##### Informações básicas dos dados

In [ ]:
print("--- Quantidade de caracteres ---")
total = 0
tamanho = 0
for i in df1.ReviewText:
    tamanho = tamanho + len(i)
total = total + tamanho
print("Gun: ", tamanho)
print("Média: ", tamanho/len(df1.ReviewText))

tamanho = 0
for i in df2.ReviewText:
    tamanho = tamanho + len(i)
total = total + tamanho
print("\nGay: ", tamanho)
print("Média: ", tamanho/len(df2.ReviewText))

tamanho = 0
for i in df3.ReviewText:
    tamanho = tamanho + len(i)
total = total + tamanho
print("\nRestaurante: ", tamanho)
print("Média: ", tamanho/len(df3.ReviewText))
    
print("\nTotal: ", total)
print("Média: ", total/(len(df1.ReviewText)+len(df2.ReviewText)+len(df3.ReviewText)))



print("\n\n--- Quantidade de textos ---")
print("Gun: ", len(df1))
print("Gay: ", len(df2))
print("Restaurante: ", len(df3))
print("Total: ", len(df1)+len(df2)+len(df3))

##### Verifica se o problema de ter texto em campo de caracteres foi corrigido

In [ ]:
# Verifica se há texto em outros campos

def tamanho(x):
    return len(x)

print(sum([df1["ReviewMeta"][df1["ReviewMeta"].apply(tamanho) > 27].size,df2["ReviewMeta"][df2["ReviewMeta"].apply(tamanho) > 27].size,df3["ReviewMeta"][df3["ReviewMeta"].apply(tamanho) > 27].size]))
print(sum([df1[0][df1[0].apply(tamanho) > 27].size,df2[0][df2[0].apply(tamanho) > 29].size,df3[0][df3[0].apply(tamanho) > 27].size]))

### Datasets 1.6MM

In [ ]:
def millions_dataset(range_inicio, range_fim):
    count = 0
    linhas = []

    # Tratamentos básicos
    for i in tqdm_notebook(range(range_inicio, range_fim)):
        if path.exists("../Datasets/Originais/Keystrokes/files/" + str(i) + "_keystrokes.txt"):
            file = open("../Datasets/Originais/Keystrokes/files/" + str(i) + "_keystrokes.txt", "r")
            count += 1

            for linha in file:
                linha = linha.replace("\n","")
                linha = linha.split("\t")
                if len(linha) > 9:
                    linha[8] = linha[9]
                    linha.pop()
                if not (linha[0] == 'PARTICIPANT_ID' and count > 1):
                    linhas.append(linha)

    dados = pd.DataFrame(data=linhas[1:], columns=linhas[0])

    
    # Converter para o formato padrão
    amostras = []    # lista para salvar as amostras tratadas
    qtd_cols = 0     # definir a qtd de colunas (maior amostra)
    for user in tqdm_notebook(dados.PARTICIPANT_ID.unique()):
        aux1 = dados[dados.PARTICIPANT_ID == user]

        for section in aux1.TEST_SECTION_ID.unique():
            aux2 = aux1[aux1.TEST_SECTION_ID == section]

            infos = list(aux2.iloc[0, 0:3])
            keys = []
            for i in range(len(aux2)):
                keys.append(str(aux2.iloc[i, 5]) + " KeyDown " + str(aux2.iloc[i, 8]))
                keys.append(str(aux2.iloc[i, 6]) + " KeyUp " + str(aux2.iloc[i, 8]))

            keys.sort()
            amostra = infos + keys
            if len(amostra) > qtd_cols:
                qtd_cols = len(infos + keys)

            amostras.append(amostra)

    df = pd.DataFrame(amostras, columns=['user_ID', 'section', 'sentence'] + list(range(qtd_cols-3)))
    
    # Cria campos user_label e sample_label
    for user in df["user_ID"].unique():
        if (df["user_ID"] == user).sum() != 15:
            print("AAA")
            df = df[df["user_ID"] != user]
    df["User_Label"] = df["user_ID"].astype(int).astype(str)
    df["Sample_Label"] = df["user_ID"].astype(int).astype(str) + "|" + df["section"].astype(str)
    
    # Salva base tratada
    df.to_csv("../Datasets/Tratados/keystrokes/keystrokes_" + str(range_inicio) + "_" + str(range_fim) + ".csv")
    print("Qtd. Usuários:", count)
    return df

for i in range(0,48):
    df = millions_dataset(10000*i, 10000*(i+1))
    print(df.shape)

In [ ]:
lista_datasets = []
for i in range(10):
    lista_datasets.append(pd.read_csv("../Datasets/Tratados/keystrokes/keystrokes_" + str(i*10000) + "_" + str((i+1)*10000) + ".csv", low_memory=False))

dataset_full = pd.concat(lista_datasets)
dataset_full.to_csv("../Datasets/Tratados/keystrokes/keystrokes_FULL.csv")

In [ ]:
dataset_full.shape

#### Agrupa dataset 15 -> 5 amostras (3 -> 1 amostra) 

In [ ]:
big = pd.read_csv("../Datasets/Tratados/keystrokes/keystrokes_0_10000.csv", index=False)

# inicialização
i = 0

fim_1 = 0
for j in big.iloc[i, :-2]:
    if j == "":
        break
    fim_1+=1

fim_2 = 0
for j in big.iloc[i+1, :-2]:
    if j == "":
        break
    fim_2+=1

fim_3 = 0
for j in big.iloc[i+2, :-2]:
    if j == "":
        break
    fim_3+=1

dados = pd.concat([big.iloc[i, :fim_1], big.iloc[i+1, :fim_2], big.iloc[i+2, :fim_3]])
label = big.iloc[i, [-2, -1]]

df_modelo = pd.DataFrame(dados).reset_index(drop=True).transpose()
df_modelo["User_Label"] = label[0]
df_modelo["Sample_Label"] = label[1]

for i in range(3,len(big),3):
    fim_1 = 0
    for j in big.iloc[i, :-2]:
        if j == "":
            break
        fim_1+=1

    fim_2 = 0
    for j in big.iloc[i+1, :-2]:
        if j == "":
            break
        fim_2+=1

    fim_3 = 0
    for j in big.iloc[i+2, :-2]:
        if j == "":
            break
        fim_3+=1

    dados = pd.concat([big.iloc[i, :fim_1], big.iloc[i+1, :fim_2], big.iloc[i+2, :fim_3]])
    label = big.iloc[i, [-2, -1]]
    
    df_aux = pd.DataFrame(dados).reset_index(drop=True).transpose()
    df_aux["User_Label"] = label[0]
    df_aux["Sample_Label"] = label[1]
    
    df_modelo = pd.concat([df_modelo, df_aux])
    
df_modelo.to_csv("../Datasets/Tratados/keystrokes/keystrokes_0_10000_AGRUP_3.csv", index=False)